In [4]:
%%writefile tcpClient_pr.py
import socket
host = socket.gethostname()
port = 5000
client = socket.socket()
client.connect((host,port))
msg = input('->')
while msg.lower().strip()!='bye':
    client.send(msg.encode())
    data = client.recv(1024).decode()
    print("data recived from server : ",data)
    msg = input('->')
client.close()

Overwriting tcpClient_pr.py


In [6]:
%%writefile tcpServer_pr.py
import socket
host = socket.gethostname()
port = 5000
server = socket.socket()
server.bind((host,port))
server.listen()
conn,addr = server.accept()
print("connection from : ",str(addr))
while True:
    data = server.recv(1024).decode()
    if not data:
        break
    l = data.split()
    l = list(map(lambda x : int(x),l))
    l = list(map(lambda i : x%2!=0,l))
    l = list(map(lambda i : x**3,l))
    s=""
    for i in l:
        s += str(i)+" "
    data = s
    print("connected user : ",str(data))
    data = input('->')
    conn.send(data.encode())
conn.close()

Overwriting tcpServer_pr.py


In [7]:
%%writefile tcpServer_pr1.py
import socket
host = socket.gethostname()
port = 5000
buffer_size = 1024
with socket.socket() as s:
    s.bind((host,port))
    s.listen()
    conn,addr = s.accept()
    with conn:
        print("connected user :",addr)
        file_data = b""
        while True:
            data = conn.recv(buffer_size)
            if not data:
                break
            file_data = data
        with open('recv.ipynb','wb') as f:
            f.write(file_data)
        print('file received')

Writing tcpServer_pr1.py


In [8]:
%%writefile tcpClient_pr1.py
import socket
host = socket.gethostname()
port = 5000
buffer_size = 1024
with socket.socket() as s:
    s.connect((host,port))
    with open('you.ipynb','rb') as f:
        while True:
            data = f.read(buffer_size)
            if not data:
                break
            s.sendAll(data)
        print('file sent')

Writing tcpClient_pr1.py


In [20]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

with open("vishal_new.html", "r", encoding="utf-8") as file:
    html_content = file.read()
    soup = BeautifulSoup(html_content)
    soup = soup.find_all("div", class_="_2kHMtA")
    len(soup)
#   print(soup)

    name_mobile = []
    rating = []
    fprize_mobile = []
    prize_mobile = []
    for i in soup:
        name_mobile.append(i.find("div", class_="_4rR01T").text)
        rating.append(float(i.find("div", class_="_3LWZlK").text))
        fprize_mobile.append(int(i.find("div", class_="_30jeq3 _1_WHN1").text[1:3] + i.find("div", class_="_30jeq3 _1_WHN1").text[4:]))
        try:
            prize_mobile.append(int(i.find("div", class_="_3I9_wc _27UcVY").text[1:].replace(",", "")))
        except:
            prize_mobile.append(np.nan)
    d = {"name_mobile": name_mobile, "rating": rating,"fprize_mobile": fprize_mobile, "prize_mobile": prize_mobile}
    df = pd.DataFrame(d)
    print(df)

                                          name_mobile  rating  fprize_mobile  \
0                    Huawei P9 (Mystic Silver, 32 GB)     4.5          39999   
1                    Huawei P9 (Prestige Gold, 32 GB)     4.5          39999   
2   SAMSUNG Galaxy S22 Plus 5G (Phantom Black, 128...     4.5          49999   
3   SAMSUNG Galaxy Z Flip3 5G (Phantom Black, 128 GB)     4.3          44999   
4   MOTOROLA Edge 30 Ultra (Interstellar Black, 25...     4.2          49999   
5    MOTOROLA Edge 30 Ultra (Starlight White, 256 GB)     4.2          49999   
6      SAMSUNG Galaxy A54 5G (Awesome Violet, 256 GB)     4.3          40999   
7                   OnePlus 8 (Glacial Green, 256 GB)     4.5          46999   
8          SAMSUNG Galaxy S22 Plus 5G (Green, 128 GB)     4.5          49999   
9               OnePlus 8 (Interstellar Glow, 256 GB)     4.5          47349   
10                  Mi 10T Pro (Cosmic Black, 128 GB)     4.2          38849   
11          SAMSUNG Galaxy Z Flip3 5G (C

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score

df = df.dropna()
x = df[["fprize_mobile","rating"]]
y = df["prize_mobile"]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
lr = LinearRegression()
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)
print(lr.coef_ )
print(lr.intercept_)

print(mean_squared_error(y_test, y_pred))
print(r2_score(y_test,y_pred))

[ 2.67628936e+00 -4.91261598e+04]
158446.25133065082
226610141.93806204
0.11614205640107444
